# 1.0 Data Collection (Web Crawling)

###### Author: Terence Tiu Chuan Jie 
###### Last Edited:9/4/2025


## 1.Newsdata io

In [1]:
from pyspark.sql import SparkSession

import sys

sys.path.append(r'/home/student/data_collected')


from newsdataapi import NewsDataApiClient
import pickle
import csv  

spark = SparkSession.builder.appName('Newsdata io').getOrCreate()

25/04/18 14:50:23 WARN Utils: Your hostname, tiu. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/18 14:50:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/18 14:50:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

api = NewsDataApiClient(apikey="pub_74815e364b4e931611590c50e5c72985ca80a")


In [4]:
import csv

class NewsFetcher:
    def __init__(self, api, country='us', category='crime', language='en'):
        self.api = api
        self.country = country
        self.category = category
        self.language = language
        self.news_list = []

    def fetch_all_news(self):
        page = None
        while True:
            try:
                response = self.api.news_api(
                    country=self.country,
                    category=self.category,
                    language=self.language,
                    page=page
                )
                news = response.get('results', [])
                self.news_list.extend(news)

                page = response.get('nextPage', None)
                if not page:
                    print("No more pages available.")
                    break
            except Exception as e:
                print(f"Error: {e}")
                break

    def get_news(self):
        return self.news_list


class NewsAnalyzer:
    def __init__(self, news_list):
        self.news_list = news_list

    def display_sample_news(self, count=5):
        for i, news in enumerate(self.news_list[:count]):
            print(f"News {i}: {news}")

    def get_all_keys(self):
        all_keys = set()
        for news in self.news_list:
            all_keys.update(news.keys())
        return all_keys


class NewsExporter:
    def __init__(self, news_list, all_keys):
        self.news_list = news_list
        self.all_keys = list(all_keys)

    def export_to_csv(self, filename='newsdata_io_14_4_test.csv'):
        try:
            with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=self.all_keys)
                writer.writeheader()
                for news in self.news_list:
                    writer.writerow(news)
            print(f"Exported {len(self.news_list)} articles to {filename}")
        except Exception as e:
            print(f"Failed to write CSV: {e}")


# === Usage ===
# `api` is already defined and connected

fetcher = NewsFetcher(api)
fetcher.fetch_all_news()

news_list = fetcher.get_news()
print(f"Total articles retrieved: {len(news_list)}")

analyzer = NewsAnalyzer(news_list)
analyzer.display_sample_news()
all_keys = analyzer.get_all_keys()
print("All keys found in articles:", all_keys)

exporter = NewsExporter(news_list, all_keys)
exporter.export_to_csv()


/tmp/ipykernel_29599/2805961141.py:15: DeprecationWarning: This method is deprecated and will be removed in upcoming updates, Instead use latest_api()
  response = self.api.news_api(


Error: {'status': 'error', 'results': {'message': 'You exceeded your assigned API credits, please check your plan and billing details.', 'code': 'ApiLimitExceeded'}}
Total articles retrieved: 190
News 0: {'article_id': '7cf5fadd91226f2920449067099412b0', 'title': 'Public Safety Logs — Monday, April 7, 2025', 'link': 'https://www.romesentinel.com/news/public-safety/public-safety-logs-monday-april-7-2025/article_fe6ead05-7304-4aa5-bc11-ba6926b5a761.html', 'keywords': None, 'creator': None, 'description': 'Rome Fire Department log', 'content': 'ONLY AVAILABLE IN PAID PLANS', 'pubDate': '2025-04-09 02:00:00', 'pubDateTZ': 'UTC', 'image_url': None, 'video_url': None, 'source_id': 'romesentinel', 'source_name': 'Romesentinel', 'source_priority': 38985, 'source_url': 'https://www.romesentinel.com', 'source_icon': 'https://i.bytvi.com/domain_icons/romesentinel.jpg', 'language': 'english', 'country': ['united states of america'], 'category': ['crime'], 'sentiment': 'ONLY AVAILABLE IN PROFESSION

In [5]:
spark.stop()

In [ ]:
# ## if u wanna use for pre- preprocessing for here some sample ideal 
# import json

# for news in news_list:
#     for key, value in news.items():
#         if isinstance(value, list):
#             news[key] = ', '.join(value) if value else None  # Convert list to string
#         elif isinstance(value, dict):
#             news[key] = json.dumps(value)  # Convert dict to JSON string


In [ ]:
# all_keys = set()
# for news in news_list:
#     all_keys.update(news.keys())

# for news in news_list:
#     for key in all_keys:
#         if key not in news:
#             news[key] = None  # Assign None to missing fields


In [ ]:
# from pyspark.sql.types import StructType, StructField, StringType, BooleanType

# schema = StructType([
#     StructField("article_id", StringType(), True),
#     StructField("title", StringType(), True),  
#     StructField("link", StringType(), True),
#     StructField("keywords", StringType(), True),
#     StructField("creator", StringType(), True),
#     StructField("video_url", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("content", StringType(), True),
#     StructField("pubDate", StringType(), True),
#     StructField("pubDateTZ", StringType(), True),
#     StructField("image_url", StringType(), True),
#     StructField("source_id", StringType(), True),
#     StructField("source_priority", StringType(), True),
#     StructField("source_name", StringType(), True),
#     StructField("source_url", StringType(), True),
#     StructField("source_icon", StringType(), True),
#     StructField("language", StringType(), True),
#     StructField("country", StringType(), True),
#     StructField("category", StringType(), True),
#     StructField("ai_tag", StringType(), True),
#     StructField("sentiment", StringType(), True),
#     StructField("sentiment_stats", StringType(), True),
#     StructField("ai_region", StringType(), True),
#     StructField("ai_org", StringType(), True),
#     StructField("duplicate", BooleanType(), True)
# ])

# df = spark.createDataFrame(news_list, schema=schema)
# df.show()


## Kafka Producer (news_io_producer.py)

In [2]:
# from kafka import KafkaProducer
# import json
# import time
# from newsdataapi import NewsDataApiClient

# api = NewsDataApiClient(apikey="pub_74815e364b4e931611590c50e5c72985ca80a")

# producer = KafkaProducer(
#     bootstrap_servers='localhost:9092',
#     value_serializer=lambda v: json.dumps(v).encode('utf-8')
# )

# def fetch_and_publish_news():
#     page = None
#     while True:
#         response = api.news_api(
#             category='crime',
#             language='en',
#             page=page
#         )
#         news_list = response.get('results', [])
#         for news in news_list:
#             # Clean and transform fields
#             new_news = {
#                 "title": news.get("title"),
#                 "content": news.get("description"),
#                 "pubDate": news.get("pubDate"),
#                 "link": news.get("link"),
#                 "category": news.get("category")
#             }
#             producer.send('news_topic', new_news)
#             print(f"Sent: {new_news.get('title')}")

#         page = response.get('nextPage', 1)
#         if not page:
#             print("No more pages. Waiting for new updates...")
#             time.sleep(60)
#         else:
#             time.sleep(2)


#     fetch_and_publish_news()


NoBrokersAvailable: NoBrokersAvailable

## for test can be stop by set the 

In [4]:
# from kafka import KafkaProducer
# import json
# import time
# from newsdataapi import NewsDataApiClient

# api = NewsDataApiClient(apikey="pub_74815e364b4e931611590c50e5c72985ca80a")

# producer = KafkaProducer(
#     bootstrap_servers='localhost:9092',
#     value_serializer=lambda v: json.dumps(v).encode('utf-8')
# )


# def fetch_and_publish_news(max_messages=None):
#     page = None
#     message_count = 0

#     while True:
#         response = api.news_api(
#             category='crime',
#             language='en',
#             page=page
#         )
#         news_list = response.get('results', [])
#         for news in news_list:
#             new_news = {
#                 "title": news.get("title"),
#                 "content": news.get("description"),
#                 "pubDate": news.get("pubDate"),
#                 "link": news.get("link"),
#                 "category": news.get("category")
#             }
#             producer.send('news_io_topic', new_news)
#             print(f"Sent: {new_news.get('title')}")
#             message_count += 1

#             # Check if we've reached max_messages (if set)
#             if max_messages and message_count >= max_messages:
#                 print(f"Reached limit of {max_messages} messages. Stopping.")
#                 producer.flush()
#                 return

#         page = response.get('nextPage', None)
#         if not page:
#             print("No more pages. Waiting for new updates...")
#             time.sleep(60)
#         else:
#             time.sleep(2)

# if __name__ == "__main__":
#     fetch_and_publish_news(max_messages=12)  # Set limit here (e.g., 20)






/tmp/ipykernel_2218/2789664699.py:19: DeprecationWarning: This method is deprecated and will be removed in upcoming updates, Instead use latest_api()
  response = api.news_api(


Sent: Two rock legends join forces and will kick off nationwide tour in Biloxi
Sent: Vyvamind: Examining the Rise of a Natural Adderall Alternative in the Cognitive Enhancement Market
Sent: INTACH Surat hosts two-day symposium on World Heritage Day
Sent: SGCCI confers lifetime achievement award on Hemant Desai
Sent: Unlicensed dog breeder made £100,000 selling puppies online
Sent: The legacy of two governors and their mausoleums
Sent: CHAGEE Rings the Opening Bell on The Nasdaq
Sent: ITS Logistics Distribution + Fulfillment Q2 Index: Conflicting Outlooks Highlight Uncertainty in Distribution and Fulfillment Space
Sent: Toll Brothers Announces New Luxury Home Community Coming Soon to Leander, Texas
Sent: Man stopped as part of PSNI anti-speeding campaign charged with importing drugs with intent to supply
Sent: Armed police stormed home due to 'risk to life' fears - but didn't find what they expected
Sent: Gray Promotes Dana Neves to Senior Managing Vice President
Reached limit of 12 mes

In [5]:
from kafka import KafkaProducer
from newsdataapi import NewsDataApiClient
import json
import time


class NewsKafkaProducer:
    def __init__(self, kafka_bootstrap_servers, kafka_topic, news_api_key):
        # Initialize Kafka producer
        self.producer = KafkaProducer(
            bootstrap_servers=kafka_bootstrap_servers,
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )

        # Initialize NewsData API client
        self.api_client = NewsDataApiClient(apikey=news_api_key)

        # Kafka topic
        self.topic = kafka_topic

    def fetch_and_publish_news(self, category='crime', language='en', max_messages=None):
        page = None
        message_count = 0

        while True:
            response = self.api_client.news_api(
                category=category,
                language=language,
                page=page
            )

            news_list = response.get('results', [])

            if not news_list:
                print("No news found. Waiting for updates...")
                time.sleep(60)
                continue

            for news in news_list:
                news_data = {
                    "title": news.get("title"),
                    "content": news.get("description"),
                    "pubDate": news.get("pubDate"),
                    "link": news.get("link"),
                    "category": news.get("category")
                }

                self.send_to_kafka(news_data)
                message_count += 1

                if max_messages and message_count >= max_messages:
                    print(f"Reached limit of {max_messages} messages. Stopping.")
                    self.producer.flush()
                    return

            page = response.get('nextPage', None)

            if not page:
                print("No more pages. Waiting for new updates...")
                time.sleep(60)
            else:
                time.sleep(2)

    def send_to_kafka(self, message):
        """Send message to Kafka topic."""
        self.producer.send(self.topic, message)
        print(f"Sent: {message.get('title')}")

    def close(self):
        """Flush and close the Kafka producer."""
        self.producer.flush()
        self.producer.close()


if __name__ == "__main__":
    kafka_servers = 'localhost:9092'
    kafka_topic = 'news_io_topic'
    news_api_key = 'pub_74815e364b4e931611590c50e5c72985ca80a'

    producer = NewsKafkaProducer(
        kafka_bootstrap_servers=kafka_servers,
        kafka_topic=kafka_topic,
        news_api_key=news_api_key
    )

    try:
        producer.fetch_and_publish_news(max_messages=12)
    finally:
        producer.close()


/tmp/ipykernel_2218/426894422.py:26: DeprecationWarning: This method is deprecated and will be removed in upcoming updates, Instead use latest_api()
  response = self.api_client.news_api(


Sent: Two rock legends join forces and will kick off nationwide tour in Biloxi
Sent: Vyvamind: Examining the Rise of a Natural Adderall Alternative in the Cognitive Enhancement Market
Sent: INTACH Surat hosts two-day symposium on World Heritage Day
Sent: SGCCI confers lifetime achievement award on Hemant Desai
Sent: Unlicensed dog breeder made £100,000 selling puppies online
Sent: The legacy of two governors and their mausoleums
Sent: CHAGEE Rings the Opening Bell on The Nasdaq
Sent: ITS Logistics Distribution + Fulfillment Q2 Index: Conflicting Outlooks Highlight Uncertainty in Distribution and Fulfillment Space
Sent: Toll Brothers Announces New Luxury Home Community Coming Soon to Leander, Texas
Sent: Man stopped as part of PSNI anti-speeding campaign charged with importing drugs with intent to supply
Sent: Armed police stormed home due to 'risk to life' fears - but didn't find what they expected
Sent: Gray Promotes Dana Neves to Senior Managing Vice President
Reached limit of 12 mes

In [6]:
from kafka import KafkaConsumer
import json
import csv

class ArticleConsumer:
    def __init__(self, kafka_server='localhost:9092', topic='news_io_topic'):
        self.consumer = KafkaConsumer(
            topic,
            bootstrap_servers=kafka_server,
            auto_offset_reset='earliest',
            # auto_offset_reset='latest',
            enable_auto_commit=True,
            value_deserializer=lambda v: json.loads(v.decode('utf-8'))
        )
        
# # Subscribe to a specific topic
# consumer.subscribe(topics=['my-topic'])
    
    def consume_and_save(self, output_file='kafka_new_io_articles_output.csv'):
        print("Consuming messages...")
        fieldnames = ['title', 'content', 'pubDate', 'link','category']  

        with open(output_file, 'w', encoding='utf-8-sig', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for message in self.consumer:
                article = message.value
                writer.writerow(article)
                print(f"Saved: {article['title'][:60]}") 

# Example usage:
consumer = ArticleConsumer()
consumer.consume_and_save()


Consuming messages...
Saved: Two rock legends join forces and will kick off nationwide to
Saved: Vyvamind: Examining the Rise of a Natural Adderall Alternati
Saved: INTACH Surat hosts two-day symposium on World Heritage Day
Saved: SGCCI confers lifetime achievement award on Hemant Desai
Saved: Unlicensed dog breeder made £100,000 selling puppies online
Saved: The legacy of two governors and their mausoleums
Saved: CHAGEE Rings the Opening Bell on The Nasdaq
Saved: ITS Logistics Distribution + Fulfillment Q2 Index: Conflicti
Saved: Toll Brothers Announces New Luxury Home Community Coming Soo
Saved: Man stopped as part of PSNI anti-speeding campaign charged w
Saved: Armed police stormed home due to 'risk to life' fears - but 
Saved: Gray Promotes Dana Neves to Senior Managing Vice President
Saved: Two rock legends join forces and will kick off nationwide to
Saved: Vyvamind: Examining the Rise of a Natural Adderall Alternati
Saved: INTACH Surat hosts two-day symposium on World Heritage Da

KeyboardInterrupt: 

In [7]:
spark.stop()